# 삼성전자 첨기연 시각 심화

- **Instructor**: Jongwoo Lim / Jiun Bae
- **Email**: [jlim@hanyang.ac.kr](mailto:jlim@hanyang.ac.kr) / [jiunbae.623@gmail.com](mailto:jiunbae.623@gmail.com)

### Import packages

First of all, Import some packages for using PyTorch.

- torch.nn: The **Network** of PyTorch basically starts with nn.Module.
- torch.nn.functional: for **Functions** such as *ReLU*, *MaxPool* (in this example)
- torch.optim: for **Optimizers**
- torchvision: Handling **Datasets**

Numpy the basic scientific computing package used in customary.

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Dataset

PyTorch basically provides CIFAR-10 Dataset and support download in running code!

In [2]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer',
		   'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
DATASET_DIR = '../data' # path to download cifar-10 dataset

TRAIN_DATASET = datasets.CIFAR10(DATASET_DIR,   # Dataset root path
                                 train=True,     # Train data
                                 download=True)  # Download if not exist

TEST_DATASET = datasets.CIFAR10(DATASET_DIR,    # Dataset root path
                                train=False)     # Test data

Extracting ../data/cifar-10-python.tar.gz to ../data


#### Network

This is a simple convolution layer network includes 3 conv layer and 2 fc layer.

In [9]:
class ConvNetwork(nn.Module):
    """Simple Neural Network contains conv layer and fc layer
    """
    def __init__(self):
        super(ConvNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Define Train and Test functions

In [10]:
from typing import Tuple

def train(model, device, train_loader, optimizer, criterion) -> float:
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    return loss.item()


def test(model, device, test_loader, criterion) -> Tuple[float, float, torch.Tensor]:
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, accuracy, output

In [11]:
torch.manual_seed(42) # 42, THE ANSWER TO LIFE, THE UNIVERSE AND EVERYTHING

batch = 64            # batch size
lr = .01              # learning rate
epochs = 10


TRAIN_DATASET.transform = transforms.ToTensor()
train_loader = torch.utils.data.DataLoader(TRAIN_DATASET,
                                           batch_size=64,
                                           shuffle=True)

TEST_DATASET.transform = transforms.ToTensor()
test_loader = torch.utils.data.DataLoader(TEST_DATASET,
                                          batch_size=64,
                                          shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
model = Network().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = F.nll_loss

for epoch in range(1, epochs + 1):
    train_loss = train(model, device, train_loader, optimizer, criterion)
    test_loss, accuracy, _ = test(model, device, test_loader, criterion)
    
    print('Epoch: {}\t Loss: {:.6f}'.format(epoch, train_loss))
    print('\t\t Average Loss: {:.4f}, Accuracy: {:.0f}%'.format(test_loss, accuracy))

torch.save(model.state_dict(), "mnist_cnn.pt")

KeyboardInterrupt: 

## Fine tuning

## Dataset

Download Dogs vs Cats dataset from [kaggle](https://www.kaggle.com/c/dogs-vs-cats/data).

In [ ]:
from pathlib import Path

from torch.utils import data
from PIL import Image

In [ ]:
class DogsCatsDataset(data.Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform

        self.images = sorted(self.root_dir.joinpath('train' if train else 'test1').glob('*.jpg'))
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx: int):
        image_file = self.images[idx]
        
        image = Image.open(str(image_file))
        label = int(image_file.name.split('.')[0] == 'dog')
        
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
DATASET_DIR = './dogs-vs-cats' # path to download dogs vs cats

TRAIN_DATASET = DogsCatsDataset(DATASET_DIR)
TEST_DATASET = DogsCatsDataset(DATASET_DIR, train=False)

TRAIN_DATASET.transform = transforms.Compose([
    transforms.RandomResizedCrop((32, 32)),
    transforms.ToTensor(),
])

TEST_DATASET.transform = transforms.Compose([
    transforms.RandomResizedCrop((32, 32)),
    transforms.ToTensor(),
])


train_loader = torch.utils.data.DataLoader(TRAIN_DATASET,
                                           batch_size=64,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(TEST_DATASET,
                                          batch_size=64,
                                          shuffle=True)

### check dataset

In [ ]:
import random
image, label = random.choice(TRAIN_DATASET)

In [ ]:
Image.fromarray((image.numpy() * 255.).astype(np.uint8).transpose(1,2,0))

In [ ]:
print (label)

## Define New Network

In [ ]:
class DCNet(nn.Module):
    def __init__(self):
        super(DCNet, self).__init__()
        self.fc1 = nn.Linear(, )
        self.fcc = nn.Linear(, )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # TODO:
        return F.log_softmax(x, dim=1)

In [ ]:
dogcat = DCNet().to(device)
optimizer = optim.SGD(dogcat.parameters(), lr=lr)
criterion = nn.BCELoss()

## get parameters from cifar-10 trained network

In [ ]:
dogcat.conv1 = model.conv1
dogcat.conv2 = model.conv2
dogcat.conv3 = model.conv3

In [ ]:
for epoch in range(1, epochs + 1):
    train_loss = train(dogcat, device, train_loader, optimizer, criterion)
    test_loss, accuracy, _ = test(dogcat, device, test_loader, criterion)
    
    print('Epoch: {}\t Loss: {:.6f}'.format(epoch, train_loss))
    print('\t\t Average Loss: {:.4f}, Accuracy: {:.0f}%'.format(test_loss, accuracy))

torch.save(model.state_dict(), "mnist_cnn.pt")